In [ ]:
from google.colab import drive
drive.mount('/content/drive') # force_remount=True

!ls "/content/drive/MyDrive"

import os
print(os.path.exists("/content/drive/MyDrive/Colab Notebooks/train.csv"))
print(os.path.exists("/content/drive/MyDrive/Colab Notebooks/test.csv"))
print(os.path.exists("/content/drive/MyDrive/Colab Notebooks/validation.csv"))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 Chat.docx
 Chat.gdoc
'ChessBoxing Notification Delivery System Architecture.gdraw'
'Colab Notebooks'
 EmiratesFareConditions.pdf
'Masters Related Document'
'MRV Fee Receipt.pdf'
'Untitled spreadsheet.gsheet'
'Visa Mock Questions Answers.gdoc'
'Visa required documents'
True
True
True


In [ ]:
# Install required libraries
!pip install --quiet transformers datasets evaluate sentence_transformers accelerate

import csv
import sys
import pandas as pd
import re
from datasets import Dataset, DatasetDict
from transformers import T5TokenizerFast

# Set random seed
import random
import numpy as np
random.seed(42)
np.random.seed(42)

# Load CSVs
def load_clean_csv(path):
    return pd.read_csv(
        path,
        quoting=3,
        sep=',',
        engine='python',
        on_bad_lines='skip',
        encoding='utf-8'
    )

csv.field_size_limit(sys.maxsize)
train_df = load_clean_csv("/content/drive/MyDrive/Colab Notebooks/train.csv")
test_df = load_clean_csv("/content/drive/MyDrive/Colab Notebooks/test.csv")
val_df = load_clean_csv("/content/drive/MyDrive/Colab Notebooks/validation.csv")

dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "test": Dataset.from_pandas(test_df),
    "val": Dataset.from_pandas(val_df)
})

print("Dataset loaded:")
print(dataset)
print("\nColumns:", dataset["train"].column_names)
print("\nFirst training example:")
print(dataset["train"][0])

# Initialize tokenizer
tokenizer = T5TokenizerFast.from_pretrained("t5-small")

def clean_text(text):
    """Remove newlines and collapse multiple spaces."""
    text = str(text).replace('\n', ' ').replace('\r', ' ')
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def truncate_at_sentence_boundary(text, tokenizer, max_length):
    """Truncate text at sentence boundary to fit within max_length tokens."""
    # Split into sentences (simple split on common sentence delimiters)
    sentences = re.split(r'(?<=[.!?])\s+', text)

    # Tokenize full text first
    tokens = tokenizer(text, truncation=False, return_tensors=None)

    # If already under limit, return as is
    if len(tokens['input_ids']) <= max_length:
        return text

    # Progressively remove sentences from end until we fit
    truncated_text = text
    for i in range(len(sentences), 0, -1):
        truncated_text = ' '.join(sentences[:i])
        tokens = tokenizer(truncated_text, truncation=False, return_tensors=None)
        if len(tokens['input_ids']) <= max_length:
            return truncated_text

    # Fallback: return first sentence if even that's too long
    return sentences[0] if sentences else text

def preprocess_function(examples):
    """Preprocess and tokenize articles and abstracts."""
    # Clean texts
    articles = [clean_text(article) for article in examples['article']]
    abstracts = [clean_text(abstract) for abstract in examples['abstract']]

    # Truncate articles at sentence boundaries
    articles = [truncate_at_sentence_boundary(article, tokenizer, 776) for article in articles]

    # Tokenize inputs
    model_inputs = tokenizer(
        articles,
        max_length=776,
        truncation=True,
        padding=False
    )

    # Tokenize targets
    labels = tokenizer(
        abstracts,
        max_length=150,
        truncation=True,
        padding=False
    )

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

# Apply preprocessing with batched processing
print("\nTokenizing datasets...")
tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset["val"].column_names,
    desc="Tokenizing"
)

print("\nTokenized dataset:")
print(tokenized_dataset)

# Save to disk
import os
save_path = "/content/drive/MyDrive/Colab Notebooks/tokenized_dataset/"
os.makedirs(save_path, exist_ok=True)
tokenized_dataset.save_to_disk(save_path)
print(f"\nTokenized dataset saved to: {save_path}")

# Print verification info
print("\n=== Dataset Counts ===")
print(f"Train: {len(tokenized_dataset['train'])} examples")
print(f"Validation: {len(tokenized_dataset['val'])} examples")
print(f"Test: {len(tokenized_dataset['test'])} examples")

print("\n=== Sample Shapes ===")
sample_train = tokenized_dataset["train"][0]
print(f"Input IDs shape: {len(sample_train['input_ids'])}")
print(f"Attention ma512sk shape: {len(sample_train['attention_mask'])}")
print(f"Labels shape: {len(sample_train['labels'])}")

print("\n=== Sample Decoded Example ===")
print("Decoded input:")
print(tokenizer.decode(sample_train['input_ids'], skip_special_tokens=True)[:300] + "...")
print("\nDecoded label:")
print(tokenizer.decode(sample_train['labels'], skip_special_tokens=True))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.2 MB/s eta 0:00:00


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/train.csv'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install any additional required packages
!pip install --quiet transformers datasets evaluate accelerate

import os
from datasets import load_from_disk
from transformers import (
    T5TokenizerFast,
    T5ForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq
)
import torch

# Set random seed for reproducibility
import random
import numpy as np
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

# Load tokenized dataset from disk
dataset_path = "/content/drive/MyDrive/Colab Notebooks/tokenized_dataset/"
print(f"Loading tokenized dataset from: {dataset_path}")
tokenized_dataset = load_from_disk(dataset_path)
print("Tokenized dataset loaded:")
print(tokenized_dataset)

# Load tokenizer and model
print("\nLoading T5-small model and tokenizer...")
tokenizer = T5TokenizerFast.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Print model parameter count
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\n=== Model Parameters ===")
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"Percentage trainable: {100 * trainable_params / total_params:.2f}%")

# Create data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True
)

# Define output directory
output_dir = "/content/drive/MyDrive/Colab Notebooks/t5_finetune_baseline"
os.makedirs(output_dir, exist_ok=True)

# Set up training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    num_train_epochs=4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    learning_rate=3e-4,
    weight_decay=0.01,
    warmup_steps=500,
    fp16=torch.cuda.is_available(),
    save_strategy="epoch",
    eval_strategy="epoch",
    save_total_limit=3,
    logging_steps=100,
    predict_with_generate=True,
    generation_max_length=150,
    generation_num_beams=4,
    load_best_model_at_end=False,
    seed=42,
    report_to="none"
)

# Placeholder compute_metrics function
def compute_metrics(eval_preds):
    """Placeholder metrics function - returns empty dict for now."""
    return {}

# Create Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["val"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

print("\n=== Training Configuration ===")
print(f"Output directory: {output_dir}")
print(f"Epochs: {training_args.num_train_epochs}")
print(f"Train batch size: {training_args.per_device_train_batch_size}")
print(f"Eval batch size: {training_args.per_device_eval_batch_size}")
print(f"Gradient accumulation steps: {training_args.gradient_accumulation_steps}")
print(f"Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"Learning rate: {training_args.learning_rate}")
print(f"Weight decay: {training_args.weight_decay}")
print(f"Warmup steps: {training_args.warmup_steps}")
print(f"FP16: {training_args.fp16}")
print(f"Generation num_beams: {training_args.generation_num_beams}")
print(f"Generation max_length: {training_args.generation_max_length}")

print("\n" + "="*60)
print("SETUP COMPLETE!")
print("="*60)
print("\n IMPORTANT: Review GPU allocation and memory before training.")
print("Check GPU status with: !nvidia-smi")
print("\n To start training, uncomment and run the following line:")
print("="*60)

# UNCOMMENT THE LINE BELOW TO START TRAINING:
# trainer.train()

In [ ]:
!pip install --quiet transformers datasets evaluate accelerate
!pip install --quiet transformers datasets evaluate sentence_transformers accelerate

import torch
import json
import pandas as pd
from datasets import load_from_disk
from transformers import (
    T5TokenizerFast,
    T5ForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq
)
import os

# Set random seed for reproducibility
import random
import numpy as np
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

# Check GPU availability
print("="*60)
print("GPU STATUS CHECK")
print("="*60)
if torch.cuda.is_available():
    print(f"✓ GPU is available!")
    print(f"Device name: {torch.cuda.get_device_name(0)}")
    print(f"Device count: {torch.cuda.device_count()}")
    print(f"Current device: {torch.cuda.current_device()}")
    print(f"Memory allocated: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")
    print(f"Memory reserved: {torch.cuda.memory_reserved(0) / 1024**3:.2f} GB")
else:
    print(" WARNING: No GPU available! Training will be very slow on CPU.")
print("="*60 + "\n")

# Define paths
dataset_path = "/content/drive/MyDrive/Colab Notebooks/tokenized_dataset/"
output_dir = "/content/drive/MyDrive/Colab Notebooks/t5_finetune_baseline"
os.makedirs(output_dir, exist_ok=True)

# Load tokenized dataset
print("Loading tokenized dataset...")
tokenized_dataset = load_from_disk(dataset_path)
print(f"Dataset loaded: {len(tokenized_dataset['train'])} train, {len(tokenized_dataset['val'])} val, {len(tokenized_dataset['test'])} test\n")

# Load tokenizer and model
print("Loading T5-small model and tokenizer...")
tokenizer = T5TokenizerFast.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")
print("Model and tokenizer loaded.\n")

# Create data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True
)

# Set up training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    num_train_epochs=4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    learning_rate=3e-4,
    weight_decay=0.01,
    warmup_steps=500,
    fp16=torch.cuda.is_available(),
    save_strategy="epoch",
    eval_strategy="epoch",
    save_total_limit=3,
    logging_steps=100,
    predict_with_generate=True,
    generation_max_length=150,
    generation_num_beams=4,
    load_best_model_at_end=False,
    seed=42,
    report_to="none"
)

# Placeholder compute_metrics function
def compute_metrics(eval_preds):
    """Placeholder metrics function - returns empty dict for now."""
    return {}

# Taking a subset of the dataset to test training
small_train = tokenized_dataset["train"].select(range(1000))
small_val = tokenized_dataset["val"].select(range(200))

# Create Seq2SeqTrainer
print("Initializing Seq2SeqTrainer...")
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["val"],
    # train_dataset=small_train,
    # eval_dataset=small_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)
print("Trainer initialized.\n")

# Start training
print("="*60)
print("STARTING TRAINING")
print("="*60)
trainer.train()
print("\n" + "="*60)
print("TRAINING COMPLETE!")
print("="*60 + "\n")

# Save model and tokenizer
print(f"Saving model to: {output_dir}")
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)
print("Model and tokenizer saved.\n")

# Save training log history
log_history_path = os.path.join(output_dir, "train_log_history.json")
print(f"Saving training log history to: {log_history_path}")
with open(log_history_path, 'w') as f:
    json.dump(trainer.state.log_history, f, indent=2)
print("Training log history saved.\n")

# Generate predictions on validation set
print("="*60)
print("GENERATING PREDICTIONS ON VALIDATION SET")
print("="*60)
print("This may take a few minutes...")

# Taking a subset of the dataset for validation
small_val = tokenized_dataset['val'].select(range(1000))

predictions_output = trainer.predict(
    tokenized_dataset['val'],
    # small_val,
    max_length=150,
    num_beams=4
)

# Decode predictions
print("Decoding predictions...")
predicted_ids = predictions_output.predictions
predicted_ids = np.where(predicted_ids != -100, predicted_ids, tokenizer.pad_token_id)
predicted_texts = tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)

# Get reference abstracts
reference_ids = predictions_output.label_ids
# Replace -100 with pad_token_id for decoding
reference_ids = np.where(reference_ids != -100, reference_ids, tokenizer.pad_token_id)
reference_texts = tokenizer.batch_decode(reference_ids, skip_special_tokens=True)

# Get original articles from validation dataset
# Need to reload original validation data
val_df = pd.read_csv(
    "drive/MyDrive/Colab Notebooks/validation.csv",
    quoting=3,
    sep=',',
    engine='python',
    on_bad_lines='skip',
    encoding='utf-8'
)

# Taking a subset of the dataset for validation
# val_df = val_df.head(1000)

# Create predictions dataframe
predictions_df = pd.DataFrame({
    'article': val_df['article'].values[:len(predicted_texts)],
    'reference_abstract': reference_texts,
    'predicted_abstract': predicted_texts
})

# Save predictions to CSV
predictions_path = os.path.join(output_dir, "val_predictions.csv")
print(f"Saving predictions to: {predictions_path}")
predictions_df.to_csv(predictions_path, index=False)
print("Predictions saved.\n")

# Print summary
print("="*60)
print("SUMMARY OF SAVED FILES")
print("="*60)
print(f"✓ Model saved to: {output_dir}")
print(f"✓ Tokenizer saved to: {output_dir}")
print(f"✓ Training log history: {log_history_path}")
print(f"✓ Validation predictions: {predictions_path}")
print(f"\nTotal validation examples: {len(predictions_df)}")
print("\nSample prediction:")
print("-" * 60)
print(f"Article (first 200 chars): {predictions_df['article'].iloc[0][:200]}...")
print(f"\nReference: {predictions_df['reference_abstract'].iloc[0]}")
print(f"\nPredicted: {predictions_df['predicted_abstract'].iloc[0]}")
print("="*60)
print("\n ALL DONE! Training and prediction complete.")

GPU STATUS CHECK

Loading tokenized dataset...
Dataset loaded: 419036 train, 401799 val, 421082 test

Loading T5-small model and tokenizer...


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Model and tokenizer loaded.

Initializing Seq2SeqTrainer...
Trainer initialized.

STARTING TRAINING


/tmp/ipython-input-1494506560.py:98: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# Install required packages
!pip install --quiet evaluate bert-score rouge-score

import pandas as pd
import numpy as np
import json
import os
import evaluate
from tqdm import tqdm

# Set random seed for reproducibility
import random
random.seed(42)
np.random.seed(42)

# Define paths
output_dir = "/content/drive/MyDrive/Colab Notebooks/t5_finetune_baseline"
predictions_path = os.path.join(output_dir, "val_predictions.csv")
metrics_path = os.path.join(output_dir, "metrics_val.json")
worst_50_path = os.path.join(output_dir, "worst_50.csv")
best_50_path = os.path.join(output_dir, "best_50.csv")

print("="*60)
print("LOADING PREDICTIONS")
print("="*60)
# Load predictions CSV
predictions_df = pd.read_csv(predictions_path)
print(f"Loaded {len(predictions_df)} predictions from: {predictions_path}")
print(f"Columns: {predictions_df.columns.tolist()}\n")

# Extract references and predictions
references = predictions_df['reference_abstract'].fillna("").astype(str).tolist()
predictions = predictions_df['predicted_abstract'].fillna("").astype(str).tolist()

# Handle edge case of empty predictions
if len(references) == 0 or len(predictions) == 0:
    print("WARNING: Empty predictions or references detected!")
    references = references if references else [""]
    predictions = predictions if predictions else [""]

print(f"Total examples: {len(references)}")
print(f"Sample reference: {references[0][:100]}...")
print(f"Sample prediction: {predictions[0][:100]}...\n")

# Load evaluation metrics
print("="*60)
print("COMPUTING ROUGE SCORES")
print("="*60)
rouge = evaluate.load("rouge")

# Compute overall ROUGE scores
rouge_results = rouge.compute(
    predictions=predictions,
    references=references,
    use_aggregator=True
)

rouge1_f1 = rouge_results['rouge1']
rouge2_f1 = rouge_results['rouge2']
rougeL_f1 = rouge_results['rougeL']

print(f"ROUGE-1 F1: {rouge1_f1:.4f}")
print(f"ROUGE-2 F1: {rouge2_f1:.4f}")
print(f"ROUGE-L F1: {rougeL_f1:.4f}\n")

# Compute per-example ROUGE-L scores for ranking
print("Computing per-example ROUGE-L scores...")
per_example_rougeL = []
for pred, ref in tqdm(zip(predictions, references), total=len(predictions), desc="ROUGE-L per example"):
    try:
        score = rouge.compute(
            predictions=[pred],
            references=[ref],
            use_aggregator=False
        )
        per_example_rougeL.append(score['rougeL'][0] if score['rougeL'] else 0.0)
    except Exception as e:
        print(f"Warning: ROUGE computation failed for one example: {e}")
        per_example_rougeL.append(0.0)

predictions_df['rouge_l_f1'] = per_example_rougeL
print(f"Per-example ROUGE-L computed. Mean: {np.mean(per_example_rougeL):.4f}\n")

# Compute BERTScore
print("="*60)
print("COMPUTING BERTSCORE")
print("="*60)
print("This may take several minutes...")

bertscore = evaluate.load("bertscore")

bertscore_results = bertscore.compute(
    predictions=predictions,
    references=references,
    model_type="bert-base-uncased",
    lang="en",
    device="cuda" if os.system("nvidia-smi > /dev/null 2>&1") == 0 else "cpu"
)

# Extract mean scores
bert_precision = np.mean(bertscore_results['precision'])
bert_recall = np.mean(bertscore_results['recall'])
bert_f1 = np.mean(bertscore_results['f1'])

print(f"BERTScore Precision: {bert_precision:.4f}")
print(f"BERTScore Recall: {bert_recall:.4f}")
print(f"BERTScore F1: {bert_f1:.4f}\n")

# Create metrics summary
metrics_summary = {
    "rouge1_f1": float(rouge1_f1),
    "rouge2_f1": float(rouge2_f1),
    "rougeL_f1": float(rougeL_f1),
    "bertscore_precision": float(bert_precision),
    "bertscore_recall": float(bert_recall),
    "bertscore_f1": float(bert_f1),
    "num_examples": len(predictions)
}

# Save metrics to JSON
print("="*60)
print("SAVING METRICS")
print("="*60)
with open(metrics_path, 'w') as f:
    json.dump(metrics_summary, f, indent=2)
print(f"✓ Metrics saved to: {metrics_path}\n")

# Print metrics table
print("="*60)
print("METRICS SUMMARY TABLE")
print("="*60)
print(f"{'Metric':<25} {'Score':<10}")
print("-" * 35)
print(f"{'ROUGE-1 F1':<25} {rouge1_f1:<10.4f}")
print(f"{'ROUGE-2 F1':<25} {rouge2_f1:<10.4f}")
print(f"{'ROUGE-L F1':<25} {rougeL_f1:<10.4f}")
print(f"{'BERTScore Precision':<25} {bert_precision:<10.4f}")
print(f"{'BERTScore Recall':<25} {bert_recall:<10.4f}")
print(f"{'BERTScore F1':<25} {bert_f1:<10.4f}")
print("="*60 + "\n")

# Sort by ROUGE-L F1 and save worst/best 50
print("="*60)
print("SAVING WORST AND BEST 50 EXAMPLES")
print("="*60)

# Sort by ROUGE-L (ascending for worst, descending for best)
sorted_worst = predictions_df.nsmallest(min(50, len(predictions_df)), 'rouge_l_f1')
sorted_best = predictions_df.nlargest(min(50, len(predictions_df)), 'rouge_l_f1')

# Save worst 50
worst_50_df = sorted_worst[['article', 'reference_abstract', 'predicted_abstract', 'rouge_l_f1']]
worst_50_df.to_csv(worst_50_path, index=False)
print(f"✓ Worst 50 examples saved to: {worst_50_path}")
print(f"  ROUGE-L range: {worst_50_df['rouge_l_f1'].min():.4f} - {worst_50_df['rouge_l_f1'].max():.4f}")

# Save best 50
best_50_df = sorted_best[['article', 'reference_abstract', 'predicted_abstract', 'rouge_l_f1']]
best_50_df.to_csv(best_50_path, index=False)
print(f"✓ Best 50 examples saved to: {best_50_path}")
print(f"  ROUGE-L range: {best_50_df['rouge_l_f1'].min():.4f} - {best_50_df['rouge_l_f1'].max():.4f}\n")

# Print sample worst and best
print("="*60)
print("SAMPLE WORST EXAMPLE")
print("="*60)
worst_example = worst_50_df.iloc[0]
print(f"ROUGE-L F1: {worst_example['rouge_l_f1']:.4f}")
print(f"Article (first 200 chars): {worst_example['article'][:200]}...")
print(f"\nReference: {worst_example['reference_abstract']}")
print(f"\nPredicted: {worst_example['predicted_abstract']}\n")

print("="*60)
print("SAMPLE BEST EXAMPLE")
print("="*60)
best_example = best_50_df.iloc[0]
print(f"ROUGE-L F1: {best_example['rouge_l_f1']:.4f}")
print(f"Article (first 200 chars): {best_example['article'][:200]}...")
print(f"\nReference: {best_example['reference_abstract']}")
print(f"\nPredicted: {best_example['predicted_abstract']}\n")

# Final summary
print("="*60)
print("SUMMARY OF SAVED FILES")
print("="*60)
print(f"✓ Metrics JSON: {metrics_path}")
print(f"✓ Worst 50 CSV: {worst_50_path}")
print(f"✓ Best 50 CSV: {best_50_path}")
print("="*60)
print("\n🎉 EVALUATION COMPLETE!")

,article,abstract
"""venous thromboembolism ( vte ) comprising of deep vein thrombosis ( dvt ) and pulmonary embolism ( pe ) can result in significant mortality",morbidity,and healthcare expenditure .
approximately,one - third of patients with symptomatic vte manifests pe,whereas two - thirds manifest dvt alone .
both dvt and pe can be clinically silent ( asymptomatic ) and hence not suspected .,None,None
if undiagnosed,asymptomatic vte can lead to chronic venous disease or recurrent vte and long - term debilitating sequelae such as postthrombotic syndrome and chronic thromboembolic pulmonary hypertension .,None
vte is not only disabling but also prolongs hospital stay and increases the cost of treatment .,None,None


Glossary part

In [ ]:
# Install required packages
!pip install --quiet transformers datasets sentence-transformers faiss-cpu tqdm

import csv
import sys
import json
import pickle
import re
import os
import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from datasets import Dataset, DatasetDict
from transformers import T5TokenizerFast
from tqdm import tqdm

# Set random seed for reproducibility
import random
random.seed(42)
np.random.seed(42)

# Define paths
glossary_faiss_path = "/content/drive/MyDrive/Colab Notebooks/glossary_faiss.index"
glossary_id_map_path = "/content/drive/MyDrive/Colab Notebooks/glossary_id_map.pkl"
glossary_emb_model_path = "/content/drive/MyDrive/Colab Notebooks/glossary_emb_model.json"
train_csv_path = "/content/drive/MyDrive/Colab Notebooks/train.csv"
val_csv_path = "/content/drive/MyDrive/Colab Notebooks/validation.csv"
test_csv_path = "/content/drive/MyDrive/Colab Notebooks/test.csv"
output_k5_path = "/content/drive/MyDrive/Colab Notebooks/tokenized_dataset_glossary_k5/"
output_k0_path = "/content/drive/MyDrive/Colab Notebooks/tokenized_dataset_glossary_k0/"

print("="*60)
print("LOADING GLOSSARY INDEX AND METADATA")
print("="*60)

# Load FAISS index
faiss_index = faiss.read_index(glossary_faiss_path)
print(f"✓ FAISS index loaded: {faiss_index.ntotal} vectors")

# Load metadata
with open(glossary_id_map_path, 'rb') as f:
    glossary_metadata = pickle.load(f)
print(f"✓ Glossary metadata loaded: {len(glossary_metadata)} entries")

# Load embedding model info
with open(glossary_emb_model_path, 'r') as f:
    model_info = json.load(f)
    embedding_model_name = model_info['model_name']
print(f"✓ Embedding model: {embedding_model_name}")

# Load embedding model
embedding_model = SentenceTransformer(embedding_model_name)
print(f"✓ Embedding model loaded\n")

# Load T5 tokenizer
tokenizer = T5TokenizerFast.from_pretrained("t5-small")
print(f"✓ T5 tokenizer loaded\n")

# Define retrieval function
def get_topk_glossary(article_text, k=5, query_len_chars=1000):
    """
    Retrieve top-k glossary entries for an article.

    Args:
        article_text: Article text
        k: Number of glossary entries to retrieve
        query_len_chars: Max chars to use from article for query

    Returns:
        List of dicts with 'term' and 'definition'
    """
    if k == 0:
        return []

    # Form query from first query_len_chars
    query_text = article_text[:query_len_chars]

    # Embed query
    query_embedding = embedding_model.encode([query_text], convert_to_numpy=True).astype('float32')

    # Search
    distances, indices = faiss_index.search(query_embedding, k)

    # Extract results
    results = []
    for idx in indices[0]:
        if idx < len(glossary_metadata):
            results.append({
                'term': glossary_metadata[idx]['term'],
                'definition': glossary_metadata[idx]['definition']
            })

    return results

# Define augmentation function
def make_augmented_text(article_text, k_entries):
    """
    Prepend glossary block to article text.

    Args:
        article_text: Original article text
        k_entries: List of glossary entries (dicts with 'term' and 'definition')

    Returns:
        Augmented text string
    """
    if len(k_entries) == 0:
        return article_text

    # Build glossary block
    glossary_parts = []
    for entry in k_entries:
        glossary_parts.append(f"{entry['term']}:{entry['definition']}")

    glossary_block = "<GLOSSARY> " + " || ".join(glossary_parts) + " </GLOSSARY>\n"

    return glossary_block + article_text

print("="*60)
print("LOADING DATASET CSVs")
print("="*60)

def load_clean_csv(path):
    return pd.read_csv(
        path,
        quoting=3,
        sep=',',
        engine='python',
        on_bad_lines='skip',
        encoding='utf-8'
    )

csv.field_size_limit(sys.maxsize)
train_df = load_clean_csv(train_csv_path)
val_df = load_clean_csv(val_csv_path)
test_df = load_clean_csv(test_csv_path)

print(f"✓ Train: {len(train_df)} examples")
print(f"✓ Validation: {len(val_df)} examples")
print(f"✓ Test: {len(test_df)} examples\n")

# Text cleaning function
def clean_text(text):
    """Remove newlines and collapse multiple spaces."""
    text = str(text).replace('\n', ' ').replace('\r', ' ')
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

# Sentence-aware truncation function
def truncate_at_sentence_boundary(text, tokenizer, max_length):
    """Truncate text at sentence boundary to fit within max_length tokens."""
    sentences = re.split(r'(?<=[.!?])\s+', text)
    tokens = tokenizer(text, truncation=False, return_tensors=None)

    if len(tokens['input_ids']) <= max_length:
        return text

    for i in range(len(sentences), 0, -1):
        truncated_text = ' '.join(sentences[:i])
        tokens = tokenizer(truncated_text, truncation=False, return_tensors=None)
        if len(tokens['input_ids']) <= max_length:
            return truncated_text

    return sentences[0] if sentences else text

# Function to create augmented dataset
def create_augmented_dataset(df_dict, k):
    """
    Create augmented dataset with glossary entries.

    Args:
        df_dict: Dictionary of DataFrames {'train': df, 'val': df, 'test': df}
        k: Number of glossary entries to retrieve (0 or 5)

    Returns:
        DatasetDict with augmented articles
    """
    print(f"\n{'='*60}")
    print(f"CREATING AUGMENTED DATASET WITH k={k}")
    print(f"{'='*60}")

    augmented_dfs = {}

    for split_name, df in df_dict.items():
        print(f"\nProcessing {split_name} split...")
        augmented_articles = []

        for idx, row in tqdm(df.iterrows(), total=len(df), desc=f"Augmenting {split_name}"):
            article = clean_text(row['article'])

            # Get top-k glossary entries
            glossary_entries = get_topk_glossary(article, k=k)

            # Create augmented text
            augmented_article = make_augmented_text(article, glossary_entries)
            augmented_articles.append(augmented_article)

        # Create new dataframe
        augmented_df = df.copy()
        augmented_df['augmented_article'] = augmented_articles
        augmented_dfs[split_name] = augmented_df

        print(f"✓ {split_name}: {len(augmented_df)} augmented articles")

    return augmented_dfs

# Create both k=5 and k=0 datasets
print("="*60)
print("CREATING AUGMENTED DATASETS")
print("="*60)

df_dict = {'train': train_df, 'val': val_df, 'test': test_df}

augmented_k5 = create_augmented_dataset(df_dict, k=5)
augmented_k0 = create_augmented_dataset(df_dict, k=0)

# Tokenization function
def preprocess_function(examples):
    """Preprocess and tokenize augmented articles and abstracts."""
    # Clean abstracts
    abstracts = [clean_text(abstract) for abstract in examples['abstract']]

    # Truncate augmented articles at sentence boundaries
    articles = [truncate_at_sentence_boundary(article, tokenizer, 512)
                for article in examples['augmented_article']]

    # Tokenize inputs
    model_inputs = tokenizer(
        articles,
        max_length=512,
        truncation=True,
        padding=False
    )

    # Tokenize targets
    labels = tokenizer(
        abstracts,
        max_length=150,
        truncation=True,
        padding=False
    )

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

# Function to tokenize and save dataset
def tokenize_and_save_dataset(augmented_dfs, output_path, k_value):
    """Tokenize augmented dataset and save to disk."""
    print(f"\n{'='*60}")
    print(f"TOKENIZING DATASET WITH k={k_value}")
    print(f"{'='*60}")

    # Convert to HuggingFace Dataset
    dataset = DatasetDict({
        "train": Dataset.from_pandas(augmented_dfs['train']),
        "val": Dataset.from_pandas(augmented_dfs['val']),
        "test": Dataset.from_pandas(augmented_dfs['test'])
    })

    print(f"Dataset created:")
    print(dataset)

    # Tokenize
    print(f"\nTokenizing with batch_size=32...")
    tokenized_dataset = dataset.map(
        preprocess_function,
        batched=True,
        batch_size=32,
        remove_columns=dataset["train"].column_names,
        desc="Tokenizing"
    )

    print(f"\nTokenized dataset:")
    print(tokenized_dataset)

    # Save to disk
    os.makedirs(output_path, exist_ok=True)
    tokenized_dataset.save_to_disk(output_path)
    print(f"\n✓ Saved to: {output_path}")

    return tokenized_dataset, dataset

# Tokenize and save k=5 dataset
tokenized_k5, dataset_k5 = tokenize_and_save_dataset(augmented_k5, output_k5_path, k_value=5)

# Tokenize and save k=0 dataset
tokenized_k0, dataset_k0 = tokenize_and_save_dataset(augmented_k0, output_k0_path, k_value=0)

# Print verification info
print("\n" + "="*60)
print("VERIFICATION: DATASET COUNTS")
print("="*60)

print(f"\nk=5 Dataset:")
print(f"  Train: {len(tokenized_k5['train'])} examples")
print(f"  Validation: {len(tokenized_k5['val'])} examples")
print(f"  Test: {len(tokenized_k5['test'])} examples")

print(f"\nk=0 Dataset:")
print(f"  Train: {len(tokenized_k0['train'])} examples")
print(f"  Validation: {len(tokenized_k0['val'])} examples")
print(f"  Test: {len(tokenized_k0['test'])} examples")

# Print example from k=5
print("\n" + "="*60)
print("EXAMPLE FROM k=5 DATASET")
print("="*60)

sample_k5 = tokenized_k5["train"][0]
print(f"\nTokenized input shape: {len(sample_k5['input_ids'])} tokens")
print(f"Tokenized label shape: {len(sample_k5['labels'])} tokens")

print(f"\nDecoded augmented input (first 500 chars):")
decoded_input_k5 = tokenizer.decode(sample_k5['input_ids'], skip_special_tokens=True)
print(decoded_input_k5[:500])
print("...")

print(f"\nDecoded label:")
decoded_label_k5 = tokenizer.decode(sample_k5['labels'], skip_special_tokens=True)
print(decoded_label_k5)

# Print example from k=0
print("\n" + "="*60)
print("EXAMPLE FROM k=0 DATASET (Control)")
print("="*60)

sample_k0 = tokenized_k0["train"][0]
print(f"\nTokenized input shape: {len(sample_k0['input_ids'])} tokens")
print(f"Tokenized label shape: {len(sample_k0['labels'])} tokens")

print(f"\nDecoded input (first 500 chars):")
decoded_input_k0 = tokenizer.decode(sample_k0['input_ids'], skip_special_tokens=True)
print(decoded_input_k0[:500])
print("...")

print(f"\nDecoded label:")
decoded_label_k0 = tokenizer.decode(sample_k0['labels'], skip_special_tokens=True)
print(decoded_label_k0)

# Final summary
print("\n" + "="*60)
print("SUMMARY")
print("="*60)
print(f"✓ k=5 tokenized dataset saved to: {output_k5_path}")
print(f"✓ k=0 tokenized dataset saved to: {output_k0_path}")
print(f"\nAugmentation difference:")
print(f"  k=5 includes <GLOSSARY>...</GLOSSARY> block with 5 entries")
print(f"  k=0 is control (no glossary prepended)")
print("="*60)
print("\n🎉 AUGMENTED DATASETS CREATED AND TOKENIZED!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 53.9 MB/s eta 0:00:00
LOADING GLOSSARY INDEX AND METADATA
✓ FAISS index loaded: 389 vectors
✓ Glossary metadata loaded: 389 entries
✓ Embedding model: sentence-transformers/all-mpnet-base-v2


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

Augmenting train: 100%|██████████| 419036/419036 [1:18:42<00:00, 88.74it/s]


✓ train: 419036 augmented articles

Processing val split...


Augmenting val: 100%|██████████| 401799/401799 [1:16:27<00:00, 87.58it/s]


✓ val: 401799 augmented articles

Processing test split...


Augmenting test:  23%|██▎       | 95867/421082 [18:03<1:01:02, 88.80it/s]